# Install

* either install it locally (see below)
* or use the shared instance running on http://104.210.58.178:9200/

## Install locally

* needs Java JRE
* get ES server from https://www.elastic.co/downloads/elasticsearch, for example: 

      wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-6.2.4.zip
      unzip elasticsearch-6.2.4.zip
      cd elasticsearch-6.2.4/
      ./bin/elasticsearch (this starts the server)
      update ES_SERVER in config.py

In [1]:
from config import MY_NAME, ES_SERVER

## Is Elasticserch running?

In [14]:
import requests as req

req.get('http://{}:9200/?pretty'.format(ES_SERVER)).json()

{'cluster_name': 'mhhhhh-application',
 'cluster_uuid': 'Rp_NwCFgTtqmPR4-J0UgYQ',
 'name': 'uZexZAh',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2018-04-12T20:37:28.497551Z',
  'build_hash': 'ccec39f',
  'build_snapshot': False,
  'lucene_version': '7.2.1',
  'minimum_index_compatibility_version': '5.0.0',
  'minimum_wire_compatibility_version': '5.6.0',
  'number': '6.2.4'}}

## Indexing a document

In [15]:
doc = {
    "rank": 21,
    "city": "Boston",
    "state": "Massachusetts",
    "population2010": 617594,
    "land_area": 48.277,  
    "location": {
        "lat": 42.332,
        "lon": 71.0202 
    },
    "abbreviation": "MA",
    "description": "Boston is the capital city and most populous municipality \
    of the Commonwealth of Massachusetts in the United States. The city proper\
    covers 48 square miles (124 km2) with an estimated population of 687,584 \
    in 2017, making it also the most populous city in the New England region. \
    Boston is the seat of Suffolk County as well, although the county government\
    was disbanded on July 1, 1999."
}

# format is :  ES-url   /index name  /doctyp/doc_id 
req.post('http://{}:9200/{}_testindex/cities/21'.format(ES_SERVER, MY_NAME), json=doc).json()

{'_id': '21',
 '_index': 'master_testindex',
 '_primary_term': 1,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'cities',
 '_version': 3,
 'result': 'updated'}

## Get the document back

In [16]:
req.get('http://{}:9200/{}_testindex/cities/21'.format(ES_SERVER, MY_NAME)).json()

{'_id': '21',
 '_index': 'master_testindex',
 '_source': {'abbreviation': 'MA',
  'city': 'Boston',
  'description': 'Boston is the capital city and most populous municipality     of the Commonwealth of Massachusetts in the United States. The city proper    covers 48 square miles (124 km2) with an estimated population of 687,584     in 2017, making it also the most populous city in the New England region.     Boston is the seat of Suffolk County as well, although the county government    was disbanded on July 1, 1999.',
  'land_area': 48.277,
  'location': {'lat': 42.332, 'lon': 71.0202},
  'population2010': 617594,
  'rank': 21,
  'state': 'Massachusetts'},
 '_type': 'cities',
 '_version': 3,
 'found': True}

## Fulltext-search a document

In [17]:
q = {  
    "query": { 
        "query_string": { 
            "query": "boston" 
        }
    }
}
req.get('http://{}:9200/{}_testindex/cities/21'.format(ES_SERVER, MY_NAME), json=q).json()

{'_id': '21',
 '_index': 'master_testindex',
 '_source': {'abbreviation': 'MA',
  'city': 'Boston',
  'description': 'Boston is the capital city and most populous municipality     of the Commonwealth of Massachusetts in the United States. The city proper    covers 48 square miles (124 km2) with an estimated population of 687,584     in 2017, making it also the most populous city in the New England region.     Boston is the seat of Suffolk County as well, although the county government    was disbanded on July 1, 1999.',
  'land_area': 48.277,
  'location': {'lat': 42.332, 'lon': 71.0202},
  'population2010': 617594,
  'rank': 21,
  'state': 'Massachusetts'},
 '_type': 'cities',
 '_version': 3,
 'found': True}

## Using Python API

Docs: http://elasticsearch-py.readthedocs.io/en/master/api.html

In [18]:
from elasticsearch import Elasticsearch,helpers
es = Elasticsearch(ES_SERVER)

In [19]:
es.search(index='{}_testindex'.format(MY_NAME), body=q)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '21',
    '_index': 'master_testindex',
    '_score': 0.39896312,
    '_source': {'abbreviation': 'MA',
     'city': 'Boston',
     'description': 'Boston is the capital city and most populous municipality     of the Commonwealth of Massachusetts in the United States. The city proper    covers 48 square miles (124 km2) with an estimated population of 687,584     in 2017, making it also the most populous city in the New England region.     Boston is the seat of Suffolk County as well, although the county government    was disbanded on July 1, 1999.',
     'land_area': 48.277,
     'location': {'lat': 42.332, 'lon': 71.0202},
     'population2010': 617594,
     'rank': 21,
     'state': 'Massachusetts'},
    '_type': 'cities'}],
  'max_score': 0.39896312,
  'total': 1},
 'timed_out': False,
 'took': 34}